In [13]:
#@title TRABAJO FINAL
import requests
import json
import datetime
import sqlite3
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
 
def disponibles():
    con = sqlite3.connect('registro_datos.db')
    cur = con.cursor()
    guardados = []
    for row in cur.execute('select distinct ticker from tick'):
        aver = {}
        for i in row:
            aver = row[0]
        guardados.append(aver)
    #print(guardados)
    print("Los tickers guardados en la base de datos son:")
    for n in guardados:
        for raw in cur.execute(f"select min(fecha) from tick where ticker='{n}'"):
            primero = raw[0]
            #print(primero)
        for raw in cur.execute(f"select max(fecha) from tick where ticker='{n}'"):
            ultimo = raw[0]
            #print(ultimo)
        if len(n) == 4:
            print(f"{n}  - {primero} <-> {ultimo}")
        elif len(n) == 3:
            print(f"{n}   - {primero} <-> {ultimo}")
        elif len(n) == 2:
            print(f"{n}    - {primero} <-> {ultimo}")
        elif len(n) == 1:
            print(f"{n}     - {primero} <-> {ultimo}")
        elif len(n) == 5:
            print(f"{n} - {primero} <-> {ultimo}")
    con.commit()
    con.close()

x = int(input('Para ACTUALIZAR LOS DATOS escriba 1\nPara VISUALIZAR LOS DATOS escriba 2\nPara ULTIMO CIERRE        escriba 3: '))

if x == 1:
    ticker_id = input("Ingrese ticker a pedir: ")
    f_ini = input('Ingrese fecha de inicio (formato YYYY-MM-DD): ')
    f_fin = input('Ingrese fecha de    fin (formato YYYY-MM-DD): ')
    print("Pidiendo datos...")
    act_pedida = requests.get(f"https://api.polygon.io/v2/aggs/ticker/{ticker_id}/range/1/day/{f_ini}/{f_fin}?adjusted=true&sort=asc&limit=550&apiKey=Jx9K197Cci0QexVJLi_nHcu8j0JLAif3")
    data = act_pedida.json()
    #print(data)
    con = sqlite3.connect('registro_datos.db')
    cur = con.cursor()
    cur.execute('create table if not exists tick( nro integer primary key, ticker, fecha, vw, o, c, h, l, t, n, v, UNIQUE(ticker, t))')
    #print(data['results'])
    for i in data['results']:
        met = (data['ticker'], f"{(datetime.datetime.fromtimestamp(i['t'] / 1000).date())}", i['vw'], i['o'], i['c'], i['h'], i['l'], i['t'], i['n'], i['v'])
        cur.execute("INSERT OR IGNORE INTO tick(ticker, fecha, vw, o, c, h, l, t, n, v) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", met)
        con.commit()             
    con.close()
    print("Datos guardados correctamente")

elif x == 2:
    clase_pedido = int(input("Para VISUALIZAR DATOS GUARDADOS escriba 1\nPara GRAFICAR UN TICKER         escriba 2:"))
    if clase_pedido == 1:
        def ver_ticker():
            disponibles()
            ticker = input("Ingrese ticker a visualizar:")
            finicio = input("Ingrese fecha inicial (formato YYYY-MM-DD)")
            ffin = input("Ingrese fecha final (formato YYYY-MM-DD)")
            con = sqlite3.connect('registro_datos.db')
            df_sol = pd.read_sql(con=con, sql= f"SELECT * FROM tick where fecha between '{finicio}' and '{ffin}' and ticker='{ticker}' order by fecha asc")
            con.commit()
            con.close()
            pd.set_option('display.max_rows', None)
            print("REFERENCIAS\nvw = Promedio\no = Apertura\nc = Cierre\nh = Máximo\nl = Mínimo\nn= Cant. Transacciones\nv = Volúmen operado")
            print(df_sol)
        ver_ticker()
    if clase_pedido == 2:
        def graficar():
            disponibles()
            print("Si desea datos de otros ticker u otros rangos de fechas (disponibles de 2 años a la fecha) reinicie programa y elija opcion 1")
            ticker = input("Ingrese ticker a graficar:")
            finicio = input("Ingrese fecha inicial (formato YYYY-MM-DD)")
            ffin = input("Ingrese fecha final (formato YYYY-MM-DD)")
            tipo_grafico = int(input("Elementos a graficar:\nCOTIZACIONES        indique 1\nVOLUMEN OPERADO     indique 2:\nCANT. TRANSACCIONES indique 3: "))
            con = sqlite3.connect('registro_datos.db')
            df_tick = pd.read_sql(con=con, sql= f"SELECT * FROM tick where fecha between '{finicio}' and '{ffin}' and ticker='{ticker}' order by fecha asc")
            con.commit()
            con.close()
            %matplotlib inline
            cant_datos = len(df_tick)
            if cant_datos <= 100:
                rango_datos = 1
            elif cant_datos >= 101 and cant_datos < 201:
                rango_datos = 2
            elif cant_datos >= 201 and cant_datos < 280:
                rango_datos = 3
            elif cant_datos >= 280 and cant_datos < 350:
                rango_datos = 4
            elif cant_datos >= 350 and cant_datos < 420:
                rango_datos = 5
            elif cant_datos >= 420 and cant_datos < 500:
                rango_datos = 6
            else:
                rango_datos = 7
            if tipo_grafico == 1:
                fig, ax = plt.subplots(figsize = (22,8))
                plt.xticks(rotation=90)
                ax.plot(df_tick['fecha'][0::rango_datos], df_tick['c'][0::rango_datos],'b', linewidth='5', label='Cierre')
                ax.plot(df_tick['fecha'][0::rango_datos], df_tick['o'][0::rango_datos],'k', linewidth='5', label='Apertura')
                ax.plot(df_tick['fecha'][0::rango_datos], df_tick['h'][0::rango_datos],'g', linewidth='5', label='Máximo')
                ax.plot(df_tick['fecha'][0::rango_datos], df_tick['l'][0::rango_datos],'r', linewidth='5', label='Mínimo')
                ax.plot(df_tick['fecha'][0::rango_datos], df_tick['vw'][0::rango_datos],'m', linewidth='5', label='Promedio')
                ax.set_title(ticker, fontsize='xx-large')
                ax.legend(fontsize='xx-large');
            if tipo_grafico == 2:
                fig, ax = plt.subplots(figsize = (22,8))
                plt.xticks(rotation=90)
                ax.plot(df_tick['fecha'][0::rango_datos], df_tick['v'][0::rango_datos],'b', linewidth='5', label='Volumen operado')    
                ax.set_title(ticker, fontsize='xx-large')
                ax.legend(fontsize='xx-large');
            if tipo_grafico == 3:
                fig, ax = plt.subplots(figsize = (22,8))
                plt.xticks(rotation=90)
                ax.plot(df_tick['fecha'][0::rango_datos], df_tick['n'][0::rango_datos],'g', linewidth='5', label='Transacciones')    
                ax.set_title(ticker, fontsize='xx-large')
                ax.legend(fontsize='xx-large');
        graficar()        


elif x == 3:
    ticker_id = input("Ingrese ticker a pedir: ")
    ultimo_cierre = requests.get(f"https://api.polygon.io/v2/aggs/ticker/{ticker_id}/prev?adjusted=true&apiKey=Jx9K197Cci0QexVJLi_nHcu8j0JLAif3")
    ultimo = ultimo_cierre.json()
    if ultimo['resultsCount'] == 0:
        print("El código de ticker ingresado es incorrecto")
    else:
        print("El último cierre de", ticker_id,"fue: U$D", ultimo['results'][0]['c'])

else:
    print("El dato ingresado es incorrecto")


Para ACTUALIZAR LOS DATOS escriba 1
Para VISUALIZAR LOS DATOS escriba 2
Para ULTIMO CIERRE        escriba 3: 3
Ingrese ticker a pedir: AAPL
El último cierre de AAPL fue: U$D 142.99
